# Initiation du modèle pourri à améliorer pour E2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
nltk.download('wordnet')
# lemmatizer pour récupérer les racines des mots
import json # pour obtenir des fonctions sur des objets JSON
import pickle # Sauvegarder un objet python

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Initialisation de mes listes 

In [ ]:
words=[] # Dictionnaire des mots
classes = [] # Dictionnaire des classes
documents = [] # Dictionnaire des patterns + tag associé
ignore_words = ['?', '!'] # Liste des mots à ignorer

# Import du fichier json
C'est à partir de ça qu'on va créer la personnalité de notre chatbot

In [ ]:
data_file = open('/content/drive/Shareddrives/P#13_Dev IA - Ecole IA Microsoft (DRIVE APPRENANTS)/P#13_ALTERNANCE/ChatBot Simplon C14/intents.json').read()

In [ ]:
intents = json.loads(data_file) 
# récupération du fichier d'intents avec des patterns, leur tag correspondant et les 
# réponses qui leur correspondent

<a id="json"></a>
# Découverte des éléments

In [ ]:
intents # Aperçu du melf intents.json

{'intents': [{'context': [''],
   'patterns': ['Je peux téléphoner ?',
    'numéro téléphone',
    'appeler',
    'appeller',
    'appel',
    'apel',
    'apelle',
    'qui',
    "comment avoir quelqu'un ?",
    "comment contacter quelqu'un?",
    'joindre',
    'contact',
    'email',
    'e-mail',
    'mail',
    'téléphone',
    'envoyer un email',
    'contacter',
    'responsable',
    'avoir un responsable',
    'chef',
    'qui est respondable?',
    'coordonnées'],
   'responses': ["Nous privilégions les échanges par mail, n'hésitez pas à nous joindre via le formulaire de contact : https://simplon.co/contact.html"],
   'tag': 'contact'},
  {'context': [''],
   'patterns': ['Distanciel ou présentiel ?',
    'Les formations sont elles disponibles en distanciel ?',
    'Les formations sont elles disponibles en présentiel ?',
    'Les formations sont elles disponibles en distanciel ?',
    'Comment les formations sont-elle dispensées ?',
    'Distance',
    'Remote',
    'Présence

# Remplissage des dossiers de travail
`words`, `documents` et `classes`

In [ ]:
for intent in intents['intents']: # pour chaque intent du fichier intents.json
    for pattern in intent['patterns']: # pour chaque pattern de chaque intent du fichier
                                        # intents.json

        #tokenize each word
        w = nltk.word_tokenize(pattern) # par exemple "Hello there" = ["hello", "there"]
        words.extend(w) # Cela ajoute chaque mot tokenizé du pattern w à la liste `words`
        #add documents in the corpus
        documents.append((w, intent['tag'])) # Le couple w et tag sont ajoutés à la liste documents
        # docs[] = [["hello", "there"],'greetings']
        # add to our classes list
        if intent['tag'] not in classes: # Si le tag n'existe pas déjà, 
            classes.append(intent['tag']) # il est ajouté à la liste des tags

In [ ]:
documents

[(['Je', 'peux', 'téléphoner', '?'], 'contact'),
 (['numéro', 'téléphone'], 'contact'),
 (['appeler'], 'contact'),
 (['appeller'], 'contact'),
 (['appel'], 'contact'),
 (['apel'], 'contact'),
 (['apelle'], 'contact'),
 (['qui'], 'contact'),
 (['comment', 'avoir', "quelqu'un", '?'], 'contact'),
 (['comment', 'contacter', "quelqu'un", '?'], 'contact'),
 (['joindre'], 'contact'),
 (['contact'], 'contact'),
 (['email'], 'contact'),
 (['e-mail'], 'contact'),
 (['mail'], 'contact'),
 (['téléphone'], 'contact'),
 (['envoyer', 'un', 'email'], 'contact'),
 (['contacter'], 'contact'),
 (['responsable'], 'contact'),
 (['avoir', 'un', 'responsable'], 'contact'),
 (['chef'], 'contact'),
 (['qui', 'est', 'respondable', '?'], 'contact'),
 (['coordonnées'], 'contact'),
 (['Distanciel', 'ou', 'présentiel', '?'], 'Mode'),
 (['Les',
   'formations',
   'sont',
   'elles',
   'disponibles',
   'en',
   'distanciel',
   '?'],
  'Mode'),
 (['Les',
   'formations',
   'sont',
   'elles',
   'disponibles',
  

In [ ]:
# Je vais réutiliser ce code donc je fais une fonction:
def display_words(display = 30):
    print(f"les {display} premiers mots sont {words[:display]}")

In [ ]:
# lemmaztize and lower each word and remove duplicates
# ça c'est une boucle compactée c'est équivalent à :
# for w in words: ## pour chaque mot de la liste words
#     if w not in ignore_words: ## si le mot n'est pas dans la liste des mots à ignorer 'ignore_words'
#        w = lemmatizer.lemmatize(w.lower()) # on prend la racine du mot sans majuscule
#        words.append(w) ## on ajout w dans la liste de mots 'words'
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

In [ ]:
words = sorted(list(set(words))) # Classement des mots dans l'ordre alphabétiques sans doublons
display_words()

les 30 premiers mots sont ['apel', 'apelle', 'appel', 'appeler', 'appeller', 'avoir', 'bjr', 'bonjour', "c'est", 'ce', 'charge', 'chef', 'combien', 'comme', 'comment', 'contact', 'contacter', 'coordonnées', 'coûte', 'cpf', 'd', 'date', 'de', 'dif', 'dispensées', 'disponibles', 'distance', 'distanciel', 'e-mail', 'elles']


In [ ]:
# On crée le bag of word
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (f"Il y a {len(documents)} documents.\n")
# classes = intents
print (f"Il y a {len(classes)} classes. \ncomposées de:\n {classes}\n")
# words = all words, vocabulary
print (f"Il y a {len(words)} mots lemmatisés sans doublons. \ncomposés de :\n {words}\n")

Il y a 60 documents.

Il y a 7 classes. 
composées de:
 ['Formation', 'Identite', 'Mode', 'Prix', 'Salutations', 'contact', 'none']

Il y a 87 mots lemmatisés sans doublons. 
composés de :
 ['apel', 'apelle', 'appel', 'appeler', 'appeller', 'avoir', 'bjr', 'bonjour', "c'est", 'ce', 'charge', 'chef', 'combien', 'comme', 'comment', 'contact', 'contacter', 'coordonnées', 'coûte', 'cpf', 'd', 'date', 'de', 'dif', 'dispensées', 'disponibles', 'distance', 'distanciel', 'e-mail', 'elles', 'email', 'en', 'envoyer', 'est', 'faites', 'financement', 'financer', 'formation', 'greeting', 'hello', 'hi', 'information', 'internet', 'je', 'joindre', 'la', 'le', 'mail', 'numéro', 'ou', 'ouverture', 'par', 'peux', 'pour', 'prix', 'prochainement', 'proposez', 'présence', 'présentiel', 'puis-je', "qu'est", "qu'est-ce-que", 'que', 'quel', 'quelle', 'quelles', "quelqu'un", 'quels', 'qui', 'quoi', 'remote', 'respondable', 'responsable', 'salut', 'simplon', 'slt', 'sont', 'sont-elle', 'sur', 'téléphone', 'télé

In [ ]:
# On exporte les dictionnaires
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# Modélisation

In [ ]:
# create our training data
training = [] # Ce sera la liste de données d'entraînement
# create an empty array for our output
output_empty = [0] * len(classes) # -> [0, 0, 0, 0, 0, 0, 0, 0, 0] 
                                # matrice qui identifiera le tag avec un nombre '1'
# training set, bag of words for each sentence
for doc in documents: # pour chaque association pattern + tag dans la liste 
                        # documents
    # un doc ressemble à : [['hi', 'there'], 'greeting']
                            # pattern , tag
                            # partie 0 , partie 1
                            # pattern , tag
    
    # list of tokenized words for the pattern
    pattern_words = doc[0] # récupération de la partie pattern d'un élément
                            # de la liste documents
    # lemmatize each word - create base word, in attempt to represent related words
    # chaque mot du pattern est mis en minuscule et on ne prend que la racine
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    
    
     # INITIALISE LE SAC DE MOTS
    bag = [] # sac de mots ce sera un vecteur avec un 1 pour chaque mot 
    #présent dans le pattern à la position du mots dans words. 
    # Donc à la fin on aura un vecteur de 88 chiffres.
    
    # REMPLISSAGE DU SAC DE MOT
    # On va comparer les mots de du pattern avec les mots de 'words'
    for w in words: # Pour chaque mot de la liste 'words'
        # La ligne qui suit est équivalente à :
        # if w in pattern_words: ## Si le mot du pattern est dans 'words'
        #    bag.append(1) ## on ajoute 1 à bag
        # else:            ## sinon
        #    bag.append(0) ## on ajoute un 0
        bag.append(1) if w in pattern_words else bag.append(0) 
        # Comme il y a 88 mots dans le dictionnaire,
        # on obtient une matrice de 88 '0' et '1' avec un '1' à la position
        # de chaque mot présent.
        # Par exemple si 'hello' et 'there' ont la position 1 et 2 dans 'words'
        #  on obtient le vecteur [1,1,0,0,...,0] un 
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    # on est toujours dans la boucle doc in documents donc on repart d'un
    # vecteur vierge pour vectoriser le tag (doc[1]) (rappel la forme de doc)
    output_row = list(output_empty) 
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row]) 
    # le doc vectorisé est ajouté au training
    # par exemple [[1,1,0,0,..,0,0][1, 0, 0, 0, 0, 0, 0, 0, 0]]
# shuffle our features and turn into np.array
random.shuffle(training) # on mélange les data d'entraînement
training = np.array(training) # on met training sous forme de tableau numpy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
training[0]

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
       list([0, 0, 0, 1, 0, 0, 0])], dtype=object)

In [ ]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0]) # on crée les données avec la colonne 0 (patterns)
train_y = list(training[:,1]) # on crée la target avec la colonne 1 (les tag)
print("Training data created")

Training data created


In [ ]:
# Create model - 
# 3 couches dense + un input
# input avec 88 neurones puisque les données d'entrées font la taille de words
# un neurone par mot de words
# une couche de 128 (par exemple) et une couche de (64)
# l'OUTPUT a autant de neurones que de classes et une fonction d'activation
# softmax car c'est une classification multiclasse
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist) # sauvegarde du model pour pouvoir
# l'utiliser ailleurs sans avoir à relancer tout ce script

print("model created")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
12/12 [==============================] - 0s 2ms/step - loss: 1.9995 - accuracy: 0.1667
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 1.8066 - accuracy: 0.4000
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 1.7297 - accuracy: 0.4000
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 1.6185 - accuracy: 0.4333
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 1.5452 - accuracy: 0.4500
Epoch 6/200
12/12 [==============================] - 0s 1ms/step - loss: 1.4973 - accuracy: 0.4667
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 1.4509 - accuracy: 0.4667
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2943 - accuracy: 0.5167
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2141 - accuracy: 0.5667
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 1.2621 - accuracy: 0.5500
Epoch 11/

# Réponse

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('/content/drive/Shareddrives/P#13_Dev IA - Ecole IA Microsoft (DRIVE APPRENANTS)/P#13_ALTERNANCE/ChatBot Simplon C14/intents.json').read()) 
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    '''
    Nettoie une phrase en la tokenisant et lemmatisant
    '''
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [ ]:
bag = [0]*9
bag

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
bag[5] = 1
bag

[0, 0, 0, 0, 0, 1, 0, 0, 0]

In [ ]:
for i, w in enumerate(words):
    print (f"L'indice c'est : {i} et le mot c'est : {w}")

L'indice c'est : 0 et le mot c'est : apel
L'indice c'est : 1 et le mot c'est : apelle
L'indice c'est : 2 et le mot c'est : appel
L'indice c'est : 3 et le mot c'est : appeler
L'indice c'est : 4 et le mot c'est : appeller
L'indice c'est : 5 et le mot c'est : avoir
L'indice c'est : 6 et le mot c'est : bjr
L'indice c'est : 7 et le mot c'est : bonjour
L'indice c'est : 8 et le mot c'est : c'est
L'indice c'est : 9 et le mot c'est : ce
L'indice c'est : 10 et le mot c'est : charge
L'indice c'est : 11 et le mot c'est : chef
L'indice c'est : 12 et le mot c'est : combien
L'indice c'est : 13 et le mot c'est : comme
L'indice c'est : 14 et le mot c'est : comment
L'indice c'est : 15 et le mot c'est : contact
L'indice c'est : 16 et le mot c'est : contacter
L'indice c'est : 17 et le mot c'est : coordonnées
L'indice c'est : 18 et le mot c'est : coûte
L'indice c'est : 19 et le mot c'est : cpf
L'indice c'est : 20 et le mot c'est : d
L'indice c'est : 21 et le mot c'est : date
L'indice c'est : 22 et le mot c

In [ ]:
def bow(sentence, words, show_details=True):
    '''
    renvoie la phrase vectorisée ex : -> [0,0,...,0] vecteur de 88
    '''
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  # -> [0,0,...,0] vecteur de 88
    for s in sentence_words: # Pour chaque mot dans la phrase
        for i,w in enumerate(words): # pour chaque mot du dico 'words'
            if w == s: # si le mot de ma phrase est présent dans 'words'
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1 # on récupère l'indice de ce mot dans 'words'
                # et on met 1 à cette position dans bag
                if show_details: # Si on a choisit l'option de montrer les détails
                    print (f"found in bag: {w}")
            else:# TODO AMELIORATION
                # Les mots qui n'ont pas été trouvé pourraient être récupérer pour améliorer le modèle
                # et le réntraîner avec les mots qu'il n'a pas sû reconnaître
                pass
    return(np.array(bag)) # retourne le sac de mots avec des 1 à la position de 
                            #chaque mot

bow renvoie ça :

In [ ]:
bow("Salut, quand est la prochaine formation à Lyon?", words,show_details=True)

found in bag: salut
found in bag: est
found in bag: la
found in bag: formation


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
test = model.predict(np.array([bow("Salut!", words,show_details=True)]))
test

found in bag: salut


array([[8.8719498e-06, 4.3523665e-05, 2.8221604e-05, 7.2530347e-05,
        9.9858677e-01, 1.5248410e-06, 1.2585046e-03]], dtype=float32)

In [ ]:
len(model.predict(np.array([bow("Salut!", words,show_details=False)]))[0])

7

In [ ]:
results = [[i,r] for i,r in enumerate(test[0]) if r>0.000005]

In [ ]:
results

[[0, 8.87195e-06],
 [1, 4.3523665e-05],
 [2, 2.8221604e-05],
 [3, 7.253035e-05],
 [4, 0.9985868],
 [6, 0.0012585046]]

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0] # Récupère les probabilités des 9 tags
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # results est les  proba qui sont supérieures au seuil 
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True) 
    # triés du plus grand au plus petit
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Test Prédiction

In [ ]:
predict_class('distenciel', model)

[{'intent': 'none', 'probability': '0.77880156'}]

In [ ]:
classes[4]

'Salutations'

### GUI

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#Creating GUI with tkinter
import tkinter # librairie d'interface graphique
from tkinter import *

In [ ]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

TclError: ignored